In [2]:
import pandas as pd

Import all the files

In [3]:
visits = pd.read_csv('visits.csv',
                     parse_dates=[1])
cart = pd.read_csv('cart.csv',
                   parse_dates=[1])
                   
checkout = pd.read_csv('checkout.csv',
                       parse_dates=[1])
purchase = pd.read_csv('purchase.csv',
                       parse_dates=[1])

Step 1: Inspect the DataFrames using `print` and `head`

In [4]:
print(visits.head(5))
print(cart.head(5))
print(checkout.head(5))
print(purchase.head(5))

                                user_id          visit_time
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00
                                user_id           cart_time
0  2be90e7c-9cca-44e0-bcc5-124b945ff168 2017-11-07 20:45:00
1  4397f73f-1da3-4ab3-91af-762792e25973 2017-05-27 01:35:00
2  a9db3d4b-0a0a-4398-a55a-ebb2c7adf663 2017-03-04 10:38:00
3  b594862a-36c5-47d5-b818-6e9512b939b3 2017-09-27 08:22:00
4  a68a16e2-94f0-4ce8-8ce3-784af0bbb974 2017-07-26 15:48:00
                                user_id       checkout_time
0  d33bdc47-4afa-45bc-b4e4-dbe948e34c0d 2017-06-25 09:29:00
1  4ac186f0-9954-4fea-8a27-c081e428e34e 2017-04-07 20:11:00
2  3c9c78a7-124a-4b77-8d2e-e1926e011e7d 2017-07-13 11:38:00
3  89fe330a-8966-4756-8f7c-3bdbcd47279a 

Step 2: Left merging visits and cart

In [5]:
visits_cart = pd.merge(visits, cart, how='left')

Step 3: How long is `visits_cart`?

In [23]:
visit_count = visits_cart.count()
print(visit_count)

user_id       2000
visit_time    2000
cart_time      348
dtype: int64


Step 4: How many timestamps are null for `cart_time`?

In [21]:
visit_only = visits_cart['cart_time'].isnull().sum()

Step 5: What percentage only visited?

In [22]:
perc_visit_only = visit_only/visit_count * 100
print(perc_visit_only)

user_id        82.600000
visit_time     82.600000
cart_time     474.712644
dtype: float64


Step 6: What percentage placed a t-shirt in their cart but did not checkout?

In [25]:
cart_checkout = pd.merge(cart, checkout, how='left')
cart_only = cart_checkout['checkout_time'].isnull().sum()
perc_check_only = cart_only / cart_checkout['checkout_time'].count() * 100
print(perc_check_only)

53.98230088495575


Step 7: Merge it all together

In [29]:
funnel_merge = visits.merge(cart, how='outer').merge(checkout, how='outer').merge(purchase, how='outer')
print(funnel_merge.head())

                                user_id          visit_time  \
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00   
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00   
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00   
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00   

            cart_time       checkout_time       purchase_time  
0                 NaT                 NaT                 NaT  
1 2017-01-26 14:44:00 2017-01-26 14:54:00 2017-01-26 15:08:00  
2 2017-08-20 08:31:00                 NaT                 NaT  
3                 NaT                 NaT                 NaT  
4                 NaT                 NaT                 NaT  


Step 8: % of users who got to checkout but did not purchase

In [35]:
check_no_purchase = funnel_merge[funnel_merge['checkout_time'].notnull() & funnel_merge['purchase_time'].isnull()]
cnp_count = check_no_purchase.count()
cnp_total = funnel_merge[funnel_merge['checkout_time'].notnull()]
cnp_perc = cnp_count / cnp_total.count() * 100
print(cnp_perc)

user_id          24.550898
visit_time       24.550898
cart_time        24.550898
checkout_time    24.550898
purchase_time     0.000000
dtype: float64


Step 9: check each part of the funnel, let's print all 3 of them again

In [36]:
print(perc_visit_only)
print(perc_check_only)
print(cnp_perc)

user_id        82.600000
visit_time     82.600000
cart_time     474.712644
dtype: float64
53.98230088495575
user_id          24.550898
visit_time       24.550898
cart_time        24.550898
checkout_time    24.550898
purchase_time     0.000000
dtype: float64


*The weakest part of the funnel is clearly getting a person who visited the site to add a tshirt to their cart. Once they've added a t-shirt to their cart it is fairly likely they end up purchasing it. A suggestion could be to make the add-to-cart button more prominent on the front page.*


Step 10: adding new column

In [46]:
funnel_merge['time_to_purchase'] = abs(funnel_merge['visit_time'] - funnel_merge['purchase_time'])

Step 11: examine the results

In [47]:
print(funnel_merge.head())

                                user_id          visit_time  \
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00   
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00   
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00   
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00   

            cart_time       checkout_time       purchase_time avg_to_purchase  \
0                 NaT                 NaT                 NaT             NaT   
1 2017-01-26 14:44:00 2017-01-26 14:54:00 2017-01-26 15:08:00 0 days 00:44:00   
2 2017-08-20 08:31:00                 NaT                 NaT             NaT   
3                 NaT                 NaT                 NaT             NaT   
4                 NaT                 NaT                 NaT             NaT   

  time_to_purchase  
0              NaT  
1  0 days 00:44:00  
2              NaT  
3              NaT  
4              NaT  


Step 12: average time to purchase

In [48]:
avg_purchase_time = funnel_merge['time_to_purchase'].mean()

In [49]:
print(avg_purchase_time)

0 days 00:43:12.380952380
